In [1]:
from icmtcp.packet.ip_packet import IPPacket
from icmtcp.packet.icmp_packet import ICMPPacket
from icmtcp.packet.tcp_packet import TCPPacket
from icmtcp.connection.ip_connection import IPConnection
from scapy.all import ICMP, IP, Raw, sniff
import socket

ModuleNotFoundError: No module named 'icmtcp'

In [2]:
packet = IPPacket()
packet.set("0.0.0.0", "127.0.0.1", "hi")

In [19]:
connection = IPConnection()
connection.open()

NameError: name 'IPConnection' is not defined

In [3]:
s = socket.socket(socket.AF_INET, socket.SOCK_RAW, socket.IPPROTO_RAW)

In [4]:
raw_packet = packet.compile()
s.sendto(raw_packet, (packet.dst_ip, 0))
recieved = s.recv(65000)

In [5]:
recieved

b'E\x00\x02\x00\x83\xcc\x00\x00@\xff\x00\x00\x7f\x00\x00\x01\x7f\x00\x00\x01hi'

In [6]:
ip_packet = IPPacket()
ip_packet.decompile(recieved)

In [7]:
print(ip_packet.src_ip, ip_packet.dst_ip, ip_packet.payload)

127.0.0.1 127.0.0.1 b'hi'


In [8]:
icmp = ICMPPacket()
icmp.dst_ip = "127.0.0.1"
icmp.code = 8
icmp.type = 8
icmp.id = 8
icmp.seq = 8
icmp.payload = "hi"

In [9]:
raw_packet = icmp.compile()
s.sendto(raw_packet, (packet.dst_ip, 0))
recieved = s.recv(65000)

In [10]:
IP(recieved)

<IP  version=4 ihl=5 tos=0x0 len=2560 id=30296 flags= frag=0 ttl=64 proto=255 chksum=0x0 src=127.0.0.1 dst=127.0.0.1 |<Raw  load=b'\x08\x08\x8f~\x00\x08\x00\x08hi' |>>

In [11]:
ICMP(IP(recieved).getlayer('Raw').load)

<ICMP  type=echo-request code=8 chksum=0x8f7e id=0x8 seq=0x8 unused=b'' |<Raw  load=b'hi' |>>

In [12]:
icmp2 = ICMPPacket()
icmp2.decompile(recieved)

In [13]:
icmp2

<IP  frag=0 proto=icmp src=127.0.0.1 dst=127.0.0.1 |<ICMP  type=echo-request code=8 id=0x8 seq=0x8 |<Raw  load=b'hi' |>>>

In [14]:
tcp = TCPPacket()
tcp.set("0.0.0.0", "127.0.0.1", "hi")
tcp.src_port = 64000
tcp.dst_port = 64001
tcp.seq = 8
tcp.ack = 8
tcp.flags = "S"

In [15]:
raw_packet = tcp.compile()
s.sendto(raw_packet, (packet.dst_ip, 0))
recieved = s.recv(65000)

In [16]:
recieved

b'E\x00\x16\x00\xc0\xb1\x00\x00@\xff\x00\x00\x7f\x00\x00\x01\x7f\x00\x00\x01\xfa\x00\xfa\x01\x00\x00\x00\x08\x00\x00\x00\x08P\x02 \x00\x00\x00\x00\x00hi'

In [17]:
tcp2 = TCPPacket()
tcp2.decompile(recieved)

In [18]:
tcp2

<IP  frag=0 proto=tcp src=127.0.0.1 dst=127.0.0.1 |<TCP  sport=64000 dport=64001 seq=8 ack=8 flags=S |<Raw  load=b'hi' |>>>